# Moses Permaul - IS362 - Project 2 - Wide Data 2


This dataset contains many fields that are not consistent. For example, the "date of birth" column should be set to DD/MM/YYYY.  The "name" column should also be fixed to have first and last name in two seperate columns. 

After cleaning this data, we will find the following:
1. Number of people in a particular state that have a college degree.
2. how many people between a certain age live in a particular state.

### Python Code for Imports and Reading the Data
To begin, we will import the standard libraries needed, read in the data, and display the first 5 rows.

In [1]:
# standard imports for numpy and pandas
import numpy as np
import pandas as pd

# import dictionary of US States from python file
from us_states import us_states

# read wide data to DataFrame
raw_data = pd.read_csv('data/wide_data_2.csv')

# make a copy of DataFrame to preserve origanal import
wide_data = raw_data.copy()

# view first 5 rows to understand the data
wide_data.head()

,Name,phonenumber,date of birth,location,Job,Degree
0,Stacy Adams,555-2219,7/23/1998,florida,NaN,no
1,"Walker, Alice",2129821112,1175,TX,lawyer,Y
2,Brandon,646-555-1236,12/18/1993,"Dallas, TX",Banker,yes
3,Robert Jones,212-999-6597,8/26/1956,NC,retired,N


### Cleaning the Data
To clean this data, we will start by looking at the column names. The column names on this DataFrame is messy because some have no spaces between words and there are no standarized capitalizations. We will start by correcting this.

We will first view the column names.

In [2]:
# view column names for the DataFrame
wide_data.columns

Index(['Name', 'phonenumber', 'date of birth', 'location', 'Job', 'Degree'], dtype='object')

To fix this, we will create a list of new column names and update the dataframe to use it.

In [3]:
# list of cleaned column names
col_names = ['Name', 'Phone Number', 'Date of Birth', 'Location', 'Job', 'Degree']

# set columns to the list created
wide_data.columns = col_names

# display the dataframe
wide_data

,Name,Phone Number,Date of Birth,Location,Job,Degree
0,Stacy Adams,555-2219,7/23/1998,florida,NaN,no
1,"Walker, Alice",2129821112,1175,TX,lawyer,Y
2,Brandon,646-555-1236,12/18/1993,"Dallas, TX",Banker,yes
3,Robert Jones,212-999-6597,8/26/1956,NC,retired,N


Looking at the data, we can see that the column "**Name**" isn't standardized across each row. We have some with both first and last names seperated by a space, last name comma first, or just a first name with no last name.

To clean this, we will first **seperate this single column of data into 2 different ones**, removing the commas, and **add it to the dataframe**. Once the column is split, we will **remove the "Name" column** as we no longer need it.

Running the following code: 

```python
wide_data[['First', 'Last']] = wide_data['Name'].str.split('_', expand=True)
```

Throws the error **ValueError: Columns must be same length as key** since splitting by a delimiter won't work on all names. To clean this data, we will need to export the data to a list, clean it, and then add it to the DataFrame as the columns "First Name" and "Last Name".

In [4]:
# export DataFrame data to list
export = wide_data['Name'].tolist()

# create lists to store first and last names
first = []
last = []

# loop through exported list
for name in export:
    
    # remove commas from name and split on whitespace
    split_names = name.replace(',', '').split()
  
    # try block to test the updating of the lists
    try:
        
        # append split names to respective lists
        first.append(split_names[0])
        last.append(split_names[1])
    
    # if there is no last name, there will be a out of range error, so append ''
    except:
        
        # append a numpy NaN value
        last.append(np.nan)

# add first and last to DataFrame
wide_data['First Name'] = first
wide_data['Last Name'] = last
        
# delete 'Name' column
del wide_data['Name']

# display the dataframe ordered for readablility
wide_data[['First Name', 'Last Name', 'Date of Birth', 'Phone Number', 'Location', 'Job', 'Degree']]

,First Name,Last Name,Date of Birth,Phone Number,Location,Job,Degree
0,Stacy,Adams,7/23/1998,555-2219,florida,NaN,no
1,Walker,Alice,1175,2129821112,TX,lawyer,Y
2,Brandon,NaN,12/18/1993,646-555-1236,"Dallas, TX",Banker,yes
3,Robert,Jones,8/26/1956,212-999-6597,NC,retired,N


Looking at the cleaned up "First Name" and "Last Name" columns, it looks like index 1 has the first and last names swapped. We'll correct this to finish up the data on names.

In [5]:
# swap column values for index 1
wide_data.loc[1][['First Name', 'Last Name']] = wide_data.loc[1][['Last Name', 'First Name']]

# display the dataframe ordered for readablility
wide_data[['First Name', 'Last Name', 'Date of Birth', 'Phone Number', 'Location', 'Job', 'Degree']]

,First Name,Last Name,Date of Birth,Phone Number,Location,Job,Degree
0,Stacy,Adams,7/23/1998,555-2219,florida,NaN,no
1,Alice,Walker,1175,2129821112,TX,lawyer,Y
2,Brandon,NaN,12/18/1993,646-555-1236,"Dallas, TX",Banker,yes
3,Robert,Jones,8/26/1956,212-999-6597,NC,retired,N


Looking at the questions we are trying to answer, we will clean up the "Date of Birth", "Locations", and "Degree" columns. These columns will allow us to analyze the data that will be required.

To start this process, we are going to convert the "Date of Birth" column to datetime with the errors paramerter set to "coerce". This will allow us to keep good data and neglect entries that are incorrect since we are unable to know what the correct values should be.

In [6]:
# convert the 'Date of Birth' column to datetime format, errors param set to 'coerce'
wide_data['Date of Birth'] = pd.to_datetime(wide_data['Date of Birth'], errors='coerce')

For the location, we will remove spaces and then do a split using the comma as a delimiter to seperate any city names from the state name. We will only keep the state portion, so we will do some checks against the imported "us_states" dictionary.

In [7]:
# export DataFrame data to list
export2 = wide_data['Location'].tolist()

# remove spaces, split each item on comma, and store the last index on each item
export2 = [x.replace(' ', '').split(',')[-1] for x in export2]

# list to store clean state/locaton data
state = []

# loop through export2 data
for x in export2:
    
    # check to see if title case version of location is in us_states keys
    if x.title() in us_states.keys():
        
        # add title case version if True 
        state.append(x.title())
    
    # check to see if upper version of location is a value in dictionary
    elif x.upper() in us_states.values():
        
        # loop through dict items
        for key, value in us_states.items(): 
            
            # check to see if upper version of location is a value
            if x.upper() == value:
                
                # add key to list
                state.append(key)

# set 'Location' column to state list    
wide_data['Location'] = state

# display the dataframe ordered for readablility
wide_data[['First Name', 'Last Name', 'Date of Birth', 'Phone Number', 'Location', 'Job', 'Degree']]

,First Name,Last Name,Date of Birth,Phone Number,Location,Job,Degree
0,Stacy,Adams,1998-07-23,555-2219,Florida,NaN,no
1,Alice,Walker,NaT,2129821112,Texas,lawyer,Y
2,Brandon,NaN,1993-12-18,646-555-1236,Texas,Banker,yes
3,Robert,Jones,1956-08-26,212-999-6597,North Carolina,retired,N


For the degree, we will clean this by keeping either a "Y" for yes and "N" for no. We will do this by exporting the upper version of the "Degree" column, and then splitting it, keeping only the 0 index.

In [8]:
# export DataFrame data to list with upper version
export3 = wide_data['Degree'].str.upper().tolist()

# split export3 and keep only the first index
split_degree = [x[0] for x in export3]

# set 'Location' column to state list    
wide_data['Degree'] = split_degree

# display the dataframe ordered for readablility
wide_data[['First Name', 'Last Name', 'Date of Birth', 'Phone Number', 'Location', 'Job', 'Degree']]

,First Name,Last Name,Date of Birth,Phone Number,Location,Job,Degree
0,Stacy,Adams,1998-07-23,555-2219,Florida,NaN,N
1,Alice,Walker,NaT,2129821112,Texas,lawyer,Y
2,Brandon,NaN,1993-12-18,646-555-1236,Texas,Banker,Y
3,Robert,Jones,1956-08-26,212-999-6597,North Carolina,retired,N


### Analyzing the Data
1. Number of people in a particular state that have a college degree.


In [9]:
# use .eq() to find degrees equal to 'Y' and group by location
wide_data['Degree'].eq('Y').astype(int).groupby(wide_data['Location']).sum()

Location
Florida           0
North Carolina    0
Texas             2
Name: Degree, dtype: int32

We can see that only North Carolina has people with degrees. There are 2 people in total.

2. How many people between a certain age live in a particular state.

In [11]:
# create copy of wide_data
df_age = wide_data.copy()

# store now timestamp
now = pd.Timestamp('now')

# create Age column based on now minus 'Date of Birth', based on Years
df_age['Age'] = (now - df_age['Date of Birth']).astype('m8[Y]')

# display the dataframe ordered for readablility
df_age[['First Name', 'Last Name', 'Date of Birth', 'Age', 'Location']]

,First Name,Last Name,Date of Birth,Age,Location
0,Stacy,Adams,1998-07-23,22.0,Florida
1,Alice,Walker,NaT,NaN,Texas
2,Brandon,NaN,1993-12-18,26.0,Texas
3,Robert,Jones,1956-08-26,64.0,North Carolina


Looking at the above DataFrame, we can now see the ages for each person as long as there was a good Date of Birth was added. We don't know exactly how the age groups will work, but now that we have the ages, we can easily group them further.

For example, we can find out how many people in each state are less than 50.

In [12]:
print('Persons Under the Age of 50:')

# use .eq() to find ages below 50 and group by location
df_age['Age'].where(df_age['Age'] < 50).groupby(df_age['Location']).count()

Persons Under the Age of 50:


Location
Florida           1
North Carolina    0
Texas             1
Name: Age, dtype: int64